In [76]:
# multinormal Native Bayes 확률 모델
# LogisticRegression : 다중클래스 회귀 기반 분류

# RidgeClassifier : 회귀 기반 분류 L2규제

# N-gram : 단점 차원폭발에 주의 (정규화/차원 축소 고려)

# kolpy Okt 

In [77]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

# 분류 모델
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [78]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
# data load
newsgroups_train = fetch_20newsgroups(
                    subset='train',
                    remove=('headers', 'footers', 'quotes'),
                    categories=categories
                  )
newsgroups_test = fetch_20newsgroups(
                    subset='test',
                    remove=('headers', 'footers', 'quotes'),
                    categories=categories
                  )


In [79]:
from sklearn.datasets import load_files
train_path = r'C:\python_src\LLM\20newsbydate\20news-bydate-train'
test_path = r'C:\python_src\LLM\20newsbydate\20news-bydate-test'

newsgroups_train = load_files(train_path, encoding='latin1')
newsgroups_test  = load_files(test_path, encoding='latin1')

In [80]:
categories

['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

In [ ]:
# 카테고리 제거
def filter_categories(dataset, categories):
    target_names = newsgroups_train.target_names
    selected_idx = [target_names.index(c) for c in categories]

    # 필터링
    data_filtered, target_filtered = [],[]
    for text, label in zip(dataset.data, dataset.target):
        if label in selected_idx:
            new_label = selected_idx.index(label)   # 라벨 재정렬
            data_filtered.append(text); target_filtered.append(new_label )
    return data_filtered, target_filtered, categories

In [82]:
train_data, train_target, target_names = filter_categories(newsgroups_train, categories)
test_data, test_target, _     = filter_categories(newsgroups_test, categories)

In [83]:
np.unique(test_target)

array([0, 1, 2, 3])

In [84]:
# 헤더 , 푸터 인용문 제거

import re

def clean_text(text):
    # 헤더 제거
    text = re.sub(r'^From:.\n', '', text, flags=re.MULTILINE)
    text = re.sub(r'^Subject:.\n', '', text, flags=re.MULTILINE)

    # 풋터 제거
    text = re.sub(r'\n--\n.$', '', text, flags=re.DOTALL)

    # 인용문 제거
    text = re.sub(r'(^|\n)[>|:].', '', text)

    return text

In [85]:
train_data = [clean_text(t) for t in train_data]
test_data  = [clean_text(t) for t in test_data ]

In [86]:
len(train_data), len(train_target),len(test_data)

(2034, 2034, 1353)

- 멀티노멀 나이즈베이즈
- 문서에 포함된 단어들의 출현 횟수를 기반으로 해서 그 문서가 어떤 주제에 속할지 확률적
- 스팸필터링, 뉴스기사 카테고리, 감성분석

- 베이즈정리 확률 이론 - 조건부 확률
- 단어A가 나왔을때 이 문서가 스팸 B 일 확률은 얼마

$P(\text{스팸} | \text{단어들}) = \frac{P(\text{단어들} | \text{스팸}) \cdot P(\text{스팸})}{P(\text{단어들})}$

- 나이브 Naive : 순진한 가정
    - 가정 : 문서 안의 모든 단어는 서로 독립적
    - 현실 : 스팸에 자주 나오는 단어들은 서로 독립적이지 않다.
    - 실제 : 이러한 가정은 계산량을 빠르게 하고 단순하지만 정확도가 어느 정도 나온다.
- 멀티노멀 : 다항분포
    - 의미 : 단어의 출현 횟수를 중요
    - 횟수를 세는 멀티노미얼 방식이 NLP 잘 맞는다.
    - 모델은 단어의 빈도수 통계
    - 스팸메일 통계
        - free : 150
        - money : 100
        - viagra : 50
        - report : 5
    - 정상메일 Ham 통계
        - report:80
        - meeting : 60
        - free : 10
    - 이러한 통계를 바탕으로 이 카테고리에서 특정 단어가 나올 확률 P('free'| 스펨)을 모두 계산
    - "Free money meeting"
        - 스팸??
            - 기본스팸확률 x, 스팸일 때 free가 나올 확률 x 스팸일 때 money가 나올 확룰 x 스팸일 때 meeting 
            나올 확률
        - 정상
            - 기본 정상확률 x 정상일 때...
            


In [87]:
target_names

# nltk tokenizer, 
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [88]:
#  min_df : 단어의 빈도가 최소 5개의 문서에 등장  - 노이즈 감소
#  max_df : 50% 너무 흔한 단어는 제거

cv = CountVectorizer(max_features=2000, min_df=5, max_df=0.5)
x_train_cv = cv.fit_transform(train_data)
x_test_cv  = cv.transform(test_data)

x_train_cv.shape, x_test_cv.shape
                

((2034, 2000), (1353, 2000))

In [89]:
cv.get_feature_names_out()

array(['00', '000', '01', ..., 'zip', 'zoo', 'zoology'],
      shape=(2000,), dtype=object)

In [90]:
x_train_cv[0].toarray()[0]

array([0, 0, 0, ..., 0, 0, 0], shape=(2000,))

In [91]:
# BOW 기반 + MNB
# 텍스트 분류의 강력한 baseline 희소데이터에 강함
nb = MultinomialNB()
# 학습용데이터 벡터데이터
nb.fit(x_train_cv, train_target)
nb.score(x_train_cv, train_target), nb.score(x_test_cv, test_target)

# 분류 리포트
from sklearn.metrics import classification_report
y_pred_nb = nb.predict(x_test_cv)
print(classification_report(test_target, y_pred_nb, target_names=categories) )

categories

                    precision    recall  f1-score   support

       alt.atheism       0.80      0.79      0.80       319
talk.religion.misc       0.74      0.73      0.73       251
     comp.graphics       0.92      0.95      0.93       389
         sci.space       0.93      0.91      0.92       394

          accuracy                           0.86      1353
         macro avg       0.85      0.85      0.85      1353
      weighted avg       0.86      0.86      0.86      1353



['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

In [92]:
# TF-IDF + MNB + LogisticRegression
# TF-IDF로 중요단어 강조, 선형모델과 자주사용 BOW 대비 흔한 단어 영향 감소
tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.5)
x_train_tfidf = tfidf.fit_transform(train_data)
x_test_tfidf  = tfidf.fit_transform(test_data)

# NB + tf-idf
nb_tfidf = MultinomialNB()
nb_tfidf.fit(x_train_tfidf, train_target)
print(nb_tfidf.score(x_train_tfidf, train_target), nb_tfidf.score(x_test_tfidf, test_target))


0.9513274336283186 0.38802660753880264


In [109]:
# logistic
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test \
    = train_test_split(x_train_tfidf, train_target, test_size=0.2
                       ,stratify=train_target, random_state=42)
lr = LogisticRegression(max_iter=1000)
lr.fit(x_train_tfidf, train_target)
print(lr.score(x_train, y_train), lr.score(x_test, y_test))

0.9834050399508297 0.9828009828009828


In [100]:
# 과적합 해결을 위한 규제
rc = RidgeClassifier(alpha=10)
rc.fit(x_train, y_train)
print(rc.score(x_train, y_train), rc.score(x_test, y_test))

0.9594345421020283 0.9336609336609336


In [102]:
# L1 규제 L1 Logistic(Lasso와 유사)
# 일부 계수를 0으로 만들어서 특성 선택을 수행.. 중요피처 select 효과
l1_lr = LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear')
l1_lr.fit(x_train, y_train)
print(l1_lr.score(x_train, y_train), l1_lr.score(x_test, y_test))

0.9373079287031346 0.914004914004914


c:\Users\khh11\miniconda3\envs\pyt_env\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(


In [115]:
# 트리 모델 + tfidf
tree = DecisionTreeClassifier(
    max_depth=5,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)
fores = RandomForestClassifier(
    n_estimators=50,     # 트리 개수
    max_depth=10,         # 최대 깊이
    min_samples_split=2,  # 분할을 위한 최소 샘플 수
    min_samples_leaf=1,   # 리프 노드의 최소 샘플 수
    random_state=42       # 재현성을 위한 시드값
)
gb = GradientBoostingClassifier(
    n_estimators=100,      # 부스팅 단계 수
    learning_rate=0.3,     # 학습률
    max_depth=5,          # 트리의 최대 깊이
    min_samples_split=2,   # 내부 노드 분할에 필요한 최소 샘플 수
    min_samples_leaf=1,    # 리프 노드가 되기 위한 최소 샘플 수
    subsample=1.0,        # 각 단계마다 사용할 샘플의 비율
    random_state=42       # 재현성을 위한 랜덤 시드
)



In [116]:
tree.fit(x_train, y_train)
print(f'tree : {tree.score(x_train, y_train), tree.score(x_test, y_test)}')

fores.fit(x_train, y_train)
print(f'fores : {fores.score(x_train, y_train), fores.score(x_test, y_test)}')

gb.fit(x_train, y_train)
print(f'gb : {gb.score(x_train, y_train), gb.score(x_test, y_test)}')


tree : (0.6385986478180701, 0.597051597051597)
fores : (0.9096496619545175, 0.8353808353808354)
fores : (1.0, 0.9434889434889435)


In [114]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 4, 5]
}

grid_search = GridSearchCV(
    GradientBoostingClassifier(),
    param_grid,
    cv=5,
    n_jobs=-1
)

grid_search.fit(x_train, y_train)
print(f"최적 파라미터: {grid_search.best_params_}")

최적 파라미터: {'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 100}


In [118]:
x_train, x_test, y_train, y_test \
    = train_test_split(train_data, train_target, test_size=0.2
                       ,stratify=train_target, random_state=42)

In [123]:
# 전처리
# RegexpTokenizer + stopwords + PosterStemmer
english_stops = set(stopwords.words('english'))
regtok = RegexpTokenizer(r"[\w']{3,}")

def custom_tokenizer(text):
    toks = regtok.tokenize(text.lower())
    toks = [t for t in toks if t not in english_stops]
    tokes = [PorterStemmer().stem(t) for t in toks]
    return tokes
tfidf_custom =TfidfVectorizer(tokenizer=custom_tokenizer, max_features=2000, min_df=5, max_df=0.5)
x_train_tfidf_c = tfidf_custom.fit_transform(x_train)
x_test_tfidf_c  = tfidf_custom.transform(x_test)

c:\Users\khh11\miniconda3\envs\pyt_env\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [124]:
from sklearn.linear_model import LogisticRegression
lr_c = LogisticRegression(max_iter=1000)
lr_c.fit(x_train_tfidf_c, y_train)
print(f'lr_c : {lr_c.score(x_train_tfidf_c, y_train), lr_c.score(x_test_tfidf_c, y_test)}')

lr_c : (0.9858635525507068, 0.9385749385749386)


In [129]:
# n-gram 실험 1,2 1,3
# 성능향상 기대 연속된 단어패턴 포착
tfidf_12 = TfidfVectorizer(token_pattern=r"[\w']{3,}"
                        , stop_words=stopwords.words('english')
                        , min_df=2, max_df=0.5
                        )

x_train_12 = tfidf_12.fit_transform(x_train)
x_test_12 = tfidf_12.transform(x_test)

lr_c = LogisticRegression(max_iter=1000)
lr_c.fit(x_train_12, y_train)

print(f'lr_c : {lr_c.score(x_train_12, y_train), lr_c.score(x_test_12, y_test)}')

lr_c : (0.9956976029502151, 0.9557739557739557)


In [132]:
# 한국어 처리 Konlpy 
# 품사 기반 태깅 tokenizer Noun Verb Adjetive
# 데이터 로딩

import pandas as pd
url = "https://drive.google.com/uc?id=1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o"
df = pd.read_csv(url)
df.head()


,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


In [133]:
df.title.unique()

array(['인피니티 워', '라라랜드', '곤지암', '신과함께', '범죄도시', '택시운전사', '코코'],
      dtype=object)

In [145]:
x_train,x_test,y_train, y_test = train_test_split(df.review, df.title, stratify=df.title, test_size=0.2, random_state=42)


In [146]:
from konlpy.tag import Okt
okt = Okt()

In [151]:
# simple version
tfidf=TfidfVectorizer(tokenizer=okt.nouns, max_features=2000, min_df=5, max_df=0.5)
x_train_tfidf=tfidf.fit_transform(x_train)
x_test_tfidf=tfidf.transform(x_test)
clf = LogisticRegression(max_iter=1000)


c:\Users\khh11\miniconda3\envs\pyt_env\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [153]:
clf.fit(x_train_tfidf, y_train)
print(clf.score(x_train_tfidf, y_train), clf.score(x_test_tfidf, y_test))

0.7574702886247878 0.6896434634974533


In [159]:
def custom_tokenizer(text):
    target = ['Noun','Verb','Adjective']
    return [w for w,tag in okt.pos(text, norm=True, stem=True) if tag in target]

tfidf=TfidfVectorizer(tokenizer=custom_tokenizer, max_features=2000, min_df=5, max_df=0.5)
x_train_tfidf=tfidf.fit_transform(x_train)
x_test_tfidf=tfidf.transform(x_test)
clf2 = LogisticRegression(max_iter=1000)
clf2.fit(x_train_tfidf, y_train)
print(clf2.score(x_train_tfidf, y_train), clf2.score(x_test_tfidf, y_test))

0.7824278438030561 0.7144312393887946
